In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
url_first_part = "https://games.crossfit.com/athlete/"

In [4]:
df_year = pd.DataFrame(data=['2017','2016','2015','2014','2013','2012','2011','2010','2009','2008'],columns=['column_A'])

try:
    del(df_main)
except:
    pass


for i in range(1,200000):
    print('page:'+str(i))
    try:
        del(df_pb)
        del(df_R)
        del(df3)
        del(open_results)
        del(df_measure)
    except:
        pass    

#     parsed_page = html.fromstring(webtext)
    try:

        url_1 = urlopen(url_first_part+str(i+100000))
        soup = BeautifulSoup(url_1)

        name = ' '.join(soup.title.string.split()[1:-3])
       
        a = 'completed page '+str(i)+' - athlete: '+name


        #region = soup.find('div',class_='text').text.strip().split('\n')
        
#get athlete profile        

        X = []
        Y = []

        for el in soup.findAll('li',class_='item sm-stacked'):

            cells = el.find_all('div',class_='item-label')
            val1=el.find_all('div', {'class': re.compile('^text lg')})

            X.append(cells[0].find(text=True))
            Y.append(val1[0].find(text=True))

        X = [item.strip() for item in X if str(item)]
        Y = [item.strip() for item in Y if str(item)]

        df_measure=pd.DataFrame(X,columns=['column_A'])
        df_measure['column_B']=Y


#get athlete workout PBs        
        
        A=[]
        B=[]

        for el in soup.findAll('table', class_='stats'):
            for row in el.find_all("tr"):

                cells = row.find_all('td')
                workout=row.findAll('th')

                A.append(cells[0].find(text=True))
                B.append(workout[0].find(text=True))


        df_pb=pd.DataFrame(B,columns=['column_A'])
        df_pb['column_B']=A

#get additional athlete
         
        E = []
        F = []



        for el in soup.find_all('li',class_='item sm-inline'):

            cells = el.find_all('div',class_='item-label')
            val1=el.find_all('div', class_='text')


            E.append(cells[0].text)
            F.append(val1[0].text)

        E = [item.strip() for item in E if str(item)]
        F = [item.strip() for item in F if str(item)]

        df_infor=pd.DataFrame(E,columns=['column_A'])
        df_infor['column_B']=F

        
        
#get Open Results        

        raw_open_results = list(filter(None,soup.find("tbody").text.strip().split('\n')))
        
        R=[]

        for y in range(0,int(len(raw_open_results)/4)):
            L=raw_open_results[y*4:(y*4)+2]
            R.append(L)
            
        df_R = pd.DataFrame(R,columns=['column_A','column_B'])
        open_results = pd.merge(df_year,df_R,how='left',on='column_A')
        
        
        df3 = df_measure.append(df_pb).append(df_infor).append(open_results)
        
        df3.set_index('column_A',inplace=True)
        df3.transpose()
        df_merged = df3.T
        #print(df_merged)
        
        df_merged['Iter_Number'] = int(i)
        df_merged.set_index('Iter_Number', inplace=True)
        
        
        #df_merged.insert(loc=0, column='Name', value=name)
        df_merged['Name']=name
        #df_merged.insert(loc=1, column='Region', value=region)
        #print(a)
        #print(df_merged)
        try: 
            try:
                df_main = df_main.append(df_merged)
            except ValueError:
                print('skipped page due to misaligned columns: '+str(i)+' - athlete: '+name)
                #print(df_merged)
                pass

        except NameError:    
            print('first athlete: '+str(i)+' - athlete: '+name)
            #print(df_merged)
            df_main = df_merged.copy()

        
        
        
    except:
        #print(df_merged)
        print('skipped page due to some missing data: '+str(i))
        pass
        #del(df_merged)
        #print(df_merged)

   

page:1


C:\Users\suhaib.qazi\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\suhaib.qazi\AppData\Local\Continuum\anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


skipped page due to some missing data: 1
page:2
first athlete: 2 - athlete: Sara du Plessis
page:3
page:4
page:5
page:6
skipped page due to some missing data: 6
page:7
page:8
page:9
skipped page due to some missing data: 9


In [5]:
df_main.to_csv(r'C:\Users\suhaib.qazi\Desktop\cf_1test.csv', sep=',')